# 實作 - CIFAR-10圖像分類
https://www.kaggle.com/c/cifar-10/


## 客製化模型

In [1]:
# 載入套件
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# CIFAR-10數據集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 資料預處理
x_train = x_train.reshape(-1, 32, 32, 3).astype('float32') / 255
x_test = x_test.reshape(-1, 32, 32, 3).astype('float32') / 255

# 將y轉為one-hot的向量
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 建立模型
model = Sequential([
    # 第一個卷積層，32個3x3的過濾器，使用ReLU激活函數
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)),
    # 最大池化層，2x2池化窗口
    MaxPooling2D(pool_size=(2, 2)),
    # 第二個卷積層，64個3x3的過濾器，使用ReLU激活函數
    Conv2D(64, kernel_size=(3, 3), activation="relu"),
    # 最大池化層，2x2池化窗口
    MaxPooling2D(pool_size=(2, 2)),
    # Dropout層，減少過擬合
    Dropout(0.5),
    # 數據扁平化層，將3D特徵轉化為1D向量
    Flatten(),
    # 全連接層，64個神經元
    Dense(64, activation="relu"),
    # Dropout層，進一步減少過擬合
    Dropout(0.5),
    # 輸出層，10個神經元，對應10個類別，使用softmax函數
    Dense(10, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 訓練模型
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_split=0.1)

# 測試模型
score = model.evaluate(x_test, y_test)
print("測試損失:", score[0])
print("測試準確度:", score[1])


170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/15
352/352 [==============================] - 14s 8ms/step - loss: 1.8371 - accuracy: 0.3203 - val_loss: 1.4684 - val_accuracy: 0.4844
Epoch 2/15
352/352 [==============================] - 2s 6ms/step - loss: 1.5399 - accuracy: 0.4405 - val_loss: 1.3190 - val_accuracy: 0.5412
Epoch 3/15
352/352 [==============================] - 2s 6ms/step - loss: 1.4427 - accuracy: 0.4780 - val_loss: 1.2345 - val_accuracy: 0.5768
Epoch 4/15
352/352 [==============================] - 2s 7ms/step - loss: 1.3839 - accuracy: 0.5041 - val_loss: 1.1789 - val_accuracy: 0.5894
Epoch 5/15
352/352 [==============================] - 3s 7ms/step - loss: 1.3298 - accuracy: 0.5280 - val_loss: 1.1424 - val_accuracy: 0.6008
Epoch 6/15
352/352 [==============================] - 2s 6ms/step - loss: 1.2878 - accuracy: 0.5431 - val_loss: 1.0938 - val_accuracy: 0.6288
Epoch 7/15
352/352 [==============================] - 2s 6ms/step - loss: 1.2556

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 6, 6, 64)          0         
                                                                 
 flatten (Flatten)           (None, 2304)              0

## 遷移學習ResNet50

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# CIFAR-10數據集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 資料預處理
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# 將y轉為one-hot的向量
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 加載預訓練的ResNet50模型
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# 讓所有參數根據新資料重新調整
base_model.trainable = True

# 建立模型
model_resnet = Sequential([
    base_model,  # 使用ResNet50作為基礎
    GlobalAveragePooling2D(),  # 使用全局平均池化
    Dropout(0.5),  # Dropout層，減少過擬合
    Dense(256, activation="relu"),  # 全連接層，256個神經元
    Dropout(0.5),  # 再次Dropout，減少過擬合
    Dense(10, activation="softmax")  # 輸出層，10個類別
])


model_resnet.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

# 訓練模型
model_resnet.fit(x_train, y_train, batch_size=128, epochs=15, validation_split=0.1)

# 測試模型
score = model_resnet.evaluate(x_test, y_test)
print("測試損失:", score[0])
print("測試準確度:", score[1])


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/15
352/352 [==============================] - 58s 75ms/step - loss: 1.6393 - accuracy: 0.4900 - val_loss: 2.6811 - val_accuracy: 0.5708
Epoch 2/15
352/352 [==============================] - 23s 64ms/step - loss: 0.9102 - accuracy: 0.7108 - val_loss: 1.0675 - val_accuracy: 0.6682
Epoch 3/15
352/352 [==============================] - 23s 66ms/step - loss: 0.7720 - accuracy: 0.7572 - val_loss: 1.6495 - val_accuracy: 0.5006
Epoch 4/15
352/352 [==============================] - 22s 64ms/step - loss: 0.8113 - accuracy: 0.7418 - val_loss: 2.0604 - val_accuracy: 0.3596
Epoch 5/15
352/352 [==============================] - 23s 64ms/step - loss: 0.7690 - accuracy: 0.7533 - val_loss: 0.8906 - val_accuracy: 0.7208
Epoch 6/15
352/352 [==============================] - 23s 66ms/step - loss: 0.5503 - accuracy: 0.8241 - val_loss: 0.6891 - val_accuracy: 0.7770
Epoch 7/15
352/352 [==============================] - 22s 64ms/step - l

In [4]:
model_resnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                      

### Pytorch版本

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torchvision.models import resnet50

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10數據集載入及預處理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 載入數據集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 切分訓練數據集為訓練和驗證集
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.1, random_state=42)
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# 載入數據加載器
train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
validation_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 建立模型
class ResNetModified(nn.Module):
    def __init__(self):
        super(ResNetModified, self).__init__()
        self.base_model = resnet50(pretrained=True)
        self.base_model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.base_model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.base_model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.base_model(x)

model = ResNetModified().to(device)

# 損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
for epoch in range(15):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # 驗證模型
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_accuracy = 100. * correct / len(validation_loader.dataset)
    print(f"Epoch {epoch+1}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%")

# 測試模型
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Validation Loss: 0.0101, Validation Accuracy: 61.86%
Epoch 2, Validation Loss: 0.0051, Validation Accuracy: 78.12%
Epoch 3, Validation Loss: 0.0053, Validation Accuracy: 78.12%
Epoch 4, Validation Loss: 0.0047, Validation Accuracy: 80.82%
Epoch 5, Validation Loss: 0.0053, Validation Accuracy: 79.88%
Epoch 6, Validation Loss: 0.0052, Validation Accuracy: 80.16%
Epoch 7, Validation Loss: 0.0052, Validation Accuracy: 80.80%
Epoch 8, Validation Loss: 0.0097, Validation Accuracy: 57.38%
Epoch 9, Validation Loss: 0.0063, Validation Accuracy: 78.66%
Epoch 10, Validation Loss: 0.0046, Validation Accuracy: 82.62%
Epoch 11, Validation Loss: 0.0051, Validation Accuracy: 82.00%
Epoch 12, Validation Loss: 0.0052, Validation Accuracy: 83.06%
Epoch 13, Validation Loss: 0.0057, Validation Accuracy: 81.68%
Epoch 14, Validation Loss: 0.0057, Validation Accuracy: 82.54%
Epoch 15, Validation Loss: 0.0056, Validation Accuracy: 82.46%
Test Loss: 0.0063, Test Accuracy: 80.72%


## Fine-tuning示範
只微調ResNet的最後5層，其他不變

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# CIFAR-10數據集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 資料預處理
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# 將y轉為one-hot的向量
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 加載預訓練的ResNet50模型
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 假設只想訓練最後5個層，其他凍住
for layer in base_model.layers[:-5]:
    layer.trainable = False

for layer in base_model.layers[-5:]:
    layer.trainable = True

# 建立模型
model_resnet = Sequential([
    base_model,  # 使用ResNet50作為基礎
    GlobalAveragePooling2D(),  # 使用全局平均池化
    Dropout(0.5),  # Dropout層，減少過擬合
    Dense(256, activation="relu"),  # 全連接層，256個神經元
    Dropout(0.5),  # 再次Dropout，減少過擬合
    Dense(10, activation="softmax")  # 輸出層，10個類別
])


model_resnet.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

# 訓練模型
model_resnet.fit(x_train, y_train, batch_size=128, epochs=15, validation_split=0.1)

# 測試模型
score = model_resnet.evaluate(x_test, y_test)
print("測試損失:", score[0])
print("測試準確度:", score[1])


Epoch 1/15
352/352 [==============================] - 23s 28ms/step - loss: 1.8000 - accuracy: 0.4606 - val_loss: 1.1527 - val_accuracy: 0.6058
Epoch 2/15
352/352 [==============================] - 7s 20ms/step - loss: 1.2756 - accuracy: 0.5538 - val_loss: 1.0615 - val_accuracy: 0.6436
Epoch 3/15
352/352 [==============================] - 7s 21ms/step - loss: 1.1704 - accuracy: 0.5911 - val_loss: 1.0109 - val_accuracy: 0.6586
Epoch 4/15
352/352 [==============================] - 8s 22ms/step - loss: 1.1001 - accuracy: 0.6150 - val_loss: 0.9839 - val_accuracy: 0.6696
Epoch 5/15
352/352 [==============================] - 8s 23ms/step - loss: 1.0568 - accuracy: 0.6293 - val_loss: 0.9693 - val_accuracy: 0.6674
Epoch 6/15
352/352 [==============================] - 8s 23ms/step - loss: 1.0084 - accuracy: 0.6452 - val_loss: 0.9458 - val_accuracy: 0.6772
Epoch 7/15
352/352 [==============================] - 7s 20ms/step - loss: 0.9761 - accuracy: 0.6553 - val_loss: 0.9330 - val_accuracy: 0.680

### Pytorch版本

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10數據集載入及預處理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 載入數據集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 切分訓練數據集為訓練和驗證集
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.1, random_state=42)
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# 載入數據加載器
train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
validation_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 建立模型
class ResNetModified(nn.Module):
    def __init__(self):
        super(ResNetModified, self).__init__()
        self.base_model = models.resnet50(pretrained=True)

        # 保留所有層，除了最後的全連接層
        self.features = nn.Sequential(*list(self.base_model.children())[:-2])

        # 添加自定义层
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(2048, 256)  # 2048 是 ResNet50 最後一個輸出特徵數
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(self.fc1(x))
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


model = ResNetModified().to(device)

# 損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# 訓練模型
for epoch in range(15):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # 驗證模型
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_accuracy = 100. * correct / len(validation_loader.dataset)
    print(f"Epoch {epoch+1}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%")

# 測試模型
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Validation Loss: 0.0076, Validation Accuracy: 68.42%
Epoch 2, Validation Loss: 0.0056, Validation Accuracy: 77.12%
Epoch 3, Validation Loss: 0.0051, Validation Accuracy: 78.78%
Epoch 4, Validation Loss: 0.0051, Validation Accuracy: 79.52%
Epoch 5, Validation Loss: 0.0045, Validation Accuracy: 81.60%
Epoch 6, Validation Loss: 0.0047, Validation Accuracy: 82.76%
Epoch 7, Validation Loss: 0.0053, Validation Accuracy: 80.90%
Epoch 8, Validation Loss: 0.0061, Validation Accuracy: 75.46%
Epoch 9, Validation Loss: 0.0051, Validation Accuracy: 81.46%
Epoch 10, Validation Loss: 0.0058, Validation Accuracy: 81.44%
Epoch 11, Validation Loss: 0.0057, Validation Accuracy: 81.14%
Epoch 12, Validation Loss: 0.0060, Validation Accuracy: 82.06%
Epoch 13, Validation Loss: 0.0062, Validation Accuracy: 81.22%
Epoch 14, Validation Loss: 0.0064, Validation Accuracy: 81.48%
Epoch 15, Validation Loss: 0.0063, Validation Accur